In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error

In [2]:
class FM:
    def __init__(self, n_factors=10, learning_rate=0.01, n_epochs=10):
        self.n_factors = n_factors
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs

    def fit(self, X, y, X_val, y_val):
        # label encoder는 1부터 카운트하기 때문에, nunique를 지정해 주기 위해 1을 더해 줌
        self.n_users = int(np.max(X[:, 0]) + 1)
        self.n_items = int(np.max(X[:, 1]) + 1)

        # 실제 y label의 평균을 global bias로 설정
        self.global_bias = np.mean(y)

        # 유저/아이템 피처별 bias를 0으로 셋팅
        self.user_bias = np.zeros(self.n_users)
        self.item_bias = np.zeros(self.n_items)

        # 평균 0, std 0.1인 정규분포에서 유저 및 아이템 팩터 초기화, 각 대상 별 n_factors개의 팩터를 할당
        self.user_factors = np.random.normal(
            scale=0.1, size=(self.n_users, self.n_factors)
        )
        self.item_factors = np.random.normal(
            scale=0.1, size=(self.n_items, self.n_factors)
        )

        for epoch in tqdm(range(self.n_epochs), desc="Training epoch"):

            # 각 epoch에 따라 다음을 수행
            for i in range(X.shape[0]):

                # 학습셋 내의 user-item pair를 대상으로 다음을 반복
                user, item = int(X[i, 0]), int(X[i, 1])

                # 예측 값 생성
                prediction = self.predict(user, item)

                # 실제 값과 예측 값을 대조함으로써 error 계산 -> loss function 계산에 사용
                e = y[i] - prediction

                # Loss function을 bias term에 대해 미분 -> (e - bias)로 계산
                self.user_bias[user] += self.learning_rate * (e - self.user_bias[user])
                self.item_bias[item] += self.learning_rate * (e - self.item_bias[item])

                # pairwise term의 Loss function을 latent factor에 대해 미분 -> (e * 상대 factor - 타겟 factor)로 계산
                self.user_factors[user] += self.learning_rate * (
                    e * self.item_factors[item] - self.user_factors[user]
                )
                self.item_factors[item] += self.learning_rate * (
                    e * self.user_factors[user] - self.item_factors[item]
                )

            # 전체 로스 계산
            loss = 0
            for i in range(X.shape[0]):
                user, item = int(X[i, 0]), int(X[i, 1])
                prediction = self.predict(user, item)
                loss += (y[i] - prediction) ** 2

            # validation pair에 대해 metrics 계산
            y_pred = [self.predict(user, item) for user, item in X_val]
            auc_score = roc_auc_score(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))

            print(
                f"Epoch {epoch + 1}/{self.n_epochs} - loss: {loss / X.shape[0]} - val_auc: {auc_score} - val_rmse: {rmse}"
            )

        return self

    def predict(self, user, item):
        # global bias와 user/item bias를 각각 더해주고, item/user factor를 내적해 준 값을 더해 최종 예측을 생성
        prediction = (
            self.global_bias
            + self.user_bias[user]
            + self.item_bias[item]
            + np.dot(self.item_factors[item], self.user_factors[user])
        )
        return prediction

In [3]:
# 데이터셋 로드
data_path = "../data/anime/"

anime = pd.read_csv(data_path + "anime.csv")
rating = pd.read_csv(data_path + "rating.csv")

In [4]:
rating_sampled = rating.sample(100_000, random_state=42)

In [5]:
# 데이터 전처리
merged = pd.merge(anime, rating_sampled, on="anime_id", suffixes=["_anime", "_user"])
merged = merged.dropna()
merged = merged[
    ["anime_id", "name", "genre", "type", "episodes", "rating_user", "user_id"]
]

# user/item id 인코딩
user_enc = LabelEncoder()
anime_enc = LabelEncoder()
merged["user"] = user_enc.fit_transform(merged["user_id"].values)
merged["anime"] = anime_enc.fit_transform(merged["anime_id"].values)

In [6]:
# 6 이상의 평점을 갖는 경우 1로, 그 외에는 0으로 인코딩
merged["rating"] = [1 if x >= 6 else 0 for x in merged["rating_user"]]

# id 정보만을 사용
X = merged[["user", "anime"]].values
y = merged["rating"].values

In [7]:
X[:5]

array([[17428,  5652],
       [ 9387,  5652],
       [21969,  5652],
       [  698,  5652],
       [ 2286,  5652]])

In [8]:
y[:5]

array([1, 1, 1, 1, 1])

In [9]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 인스턴스 생성 및 학습
fm = FM(n_factors=10, learning_rate=0.01, n_epochs=10)
fm.fit(X_train, y_train, X_val, y_val)

n_users = int(np.max(X_train[:, 0]) + 1)
n_items = int(np.max(X_train[:, 1]) + 1)

print("sparsity: ", rating_sampled.shape[0] / (n_users * n_items))

Training epoch:  10%|█         | 1/10 [00:00<00:05,  1.58it/s]

Epoch 1/10 - loss: 0.17251228138950633 - val_auc: 0.6757261649826166 - val_rmse: 0.42144339569157674


Training epoch:  20%|██        | 2/10 [00:01<00:05,  1.57it/s]

Epoch 2/10 - loss: 0.16271203355763927 - val_auc: 0.7262627548788547 - val_rmse: 0.41517354904063963


Training epoch:  30%|███       | 3/10 [00:01<00:04,  1.59it/s]

...

Epoch 8/10 - loss: 0.13076960480206096 - val_auc: 0.7997820044659742 - val_rmse: 0.3940427040597478


Training epoch:  90%|█████████ | 9/10 [00:05<00:00,  1.48it/s]

Epoch 9/10 - loss: 0.12749130364987202 - val_auc: 0.8034455973543343 - val_rmse: 0.39185787537350736


Training epoch: 100%|██████████| 10/10 [00:06<00:00,  1.53it/s]

Epoch 10/10 - loss: 0.1245361800590691 - val_auc: 0.8063895734711979 - val_rmse: 0.3898949314811094
sparsity:  0.000459066495809462


In [10]:
pd.DataFrame(X_train)[0].value_counts().mean()

2.335503459550988

In [11]:
def predict(user, item):
    # global bias와 user/item bias를 각각 더해주고, item/user factor를 내적해 준 값을 더해 최종 예측을 생성
    prediction = (
        global_bias
        + user_bias[user]
        + item_bias[item]
        + np.dot(item_factors[item], user_factors[user])
    )
    return prediction

In [12]:
n_epochs = 1
n_factors = 8
learning_rate = 0.01

# label encoder는 1부터 카운트하기 때문에, nunique를 지정해 주기 위해 1을 더해 줌
n_users = int(np.max(X[:, 0]) + 1)
n_items = int(np.max(X[:, 1]) + 1)

In [13]:
print(n_users, n_items)

37838 5757


In [14]:
# 실제 y label의 평균을 global bias로 설정
global_bias = np.mean(y)

In [15]:
global_bias

0.751085021700434

In [16]:
# 유저/아이템 피처별 bias를 0으로 셋팅
user_bias = np.zeros(n_users)
item_bias = np.zeros(n_items)

# 평균 0, std 0.1인 정규분포에서 유저 및 아이템 팩터 초기화, 각 대상 별 n_factors개의 팩터를 할당
user_factors = np.random.normal(scale=0.1, size=(n_users, n_factors))
item_factors = np.random.normal(scale=0.1, size=(n_items, n_factors))

In [17]:
user_factors[:10]

array([[ 1.16787083e-01,  3.60830616e-02,  2.89391146e-04,
        -9.24351337e-02,  2.01635089e-02, -1.00559995e-01,
         6.30785902e-02, -5.32058643e-02],
       [ 4.14997168e-02,  1.35890362e-01,  1.97700545e-02,
         5.14264723e-02,  3.31253294e-02,  1.55968229e-01,
         1.37695507e-01,  3.45388390e-02],
       [-1.84834654e-01, -5.16707354e-02, -1.60756178e-02,
        -6.09059819e-02,  3.21590321e-01, -8.66452121e-02,
         1.64674772e-02,  5.72767135e-02],
       [ 4.85516445e-02, -3.36874644e-02, -9.57123884e-03,
         1.74577169e-01,  1.38886139e-01, -1.25492445e-01,
        -1.75020843e-01,  1.61059549e-01],
       [-6.37080819e-02, -5.36736124e-02,  1.42006654e-01,
        -1.65720462e-01,  1.46010321e-01, -7.09997938e-04,
         9.70037727e-02,  4.46417650e-02],
       [-3.33368926e-02, -1.39350618e-01, -8.52916801e-02,
        -1.15827694e-02,  1.42509159e-01,  5.20440705e-02,
         2.81434865e-02,  6.41861381e-02],
       [ 9.97867176e-02,  9.337786

In [18]:
for epoch in tqdm(range(n_epochs), desc="Training epoch"):

    # 각 epoch에 따라 다음을 수행
    for i in range(X.shape[0]):

        # 학습셋 내의 user-item pair를 대상으로 다음을 반복
        user, item = int(X[i, 0]), int(X[i, 1])

        # 예측 값 생성
        prediction = predict(user, item)

        # 실제 값과 예측 값을 대조함으로써 error 계산 -> loss function 계산에 사용
        e = y[i] - prediction

        # Loss function을 bias term에 대해 미분 -> (e - bias)로 계산
        user_bias[user] += learning_rate * (e - user_bias[user])
        item_bias[item] += learning_rate * (e - item_bias[item])

        # pairwise term의 Loss function을 latent factor에 대해 미분 -> (e * 상대 factor - 타겟 factor)로 계산
        user_factors[user] += learning_rate * (
            e * item_factors[item] - user_factors[user]
        )
        item_factors[item] += learning_rate * (
            e * user_factors[user] - item_factors[item]
        )
        break

    # 전체 로스 계산
    loss = 0
    for i in range(X.shape[0]):
        user, item = int(X[i, 0]), int(X[i, 1])
        prediction = predict(user, item)
        loss += (y[i] - prediction) ** 2

    # validation pair에 대해 metric 계산
    y_pred = [predict(user, item) for user, item in X_val]
    auc_score = roc_auc_score(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    print(
        f"Epoch {epoch + 1}/{n_epochs} - loss: {loss / X.shape[0]} - val_auc: {auc_score} - val_rmse: {rmse}"
    )

Training epoch: 100%|██████████| 1/1 [00:00<00:00,  3.44it/s]

Epoch 1/1 - loss: 0.1877399559500641 - val_auc: 0.5038561224091495 - val_rmse: 0.4318023349938312


In [19]:
rating_sampled = rating.sample(1_000_000, random_state=42)

# Preprocess the data
merged = pd.merge(anime, rating_sampled, on="anime_id", suffixes=["_anime", "_user"])
merged = merged.dropna()
merged = merged[
    ["anime_id", "name", "genre", "type", "episodes", "rating_user", "user_id"]
]

# Create features from user and anime
user_enc = LabelEncoder()
anime_enc = LabelEncoder()
merged["user"] = user_enc.fit_transform(merged["user_id"].values)
merged["anime"] = anime_enc.fit_transform(merged["anime_id"].values)

# Transform ratings to binary
merged["rating"] = [1 if x >= 6 else 0 for x in merged["rating_user"]]

# Prepare the data
X = merged[["user", "anime"]].values
y = merged["rating"].values


# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Create and train the FM model
fm = FM(n_factors=10, learning_rate=0.01, n_epochs=10)
fm.fit(X_train, y_train, X_val, y_val)

n_users = int(np.max(X_train[:, 0]) + 1)
n_items = int(np.max(X_train[:, 1]) + 1)

print("sparsity: ", rating_sampled.shape[0] / (n_users * n_items))

Training epoch:  10%|█         | 1/10 [00:06<00:59,  6.58s/it]

Epoch 1/10 - loss: 0.14499846829027416 - val_auc: 0.8726614411254024 - val_rmse: 0.38639264875665236


Training epoch:  20%|██        | 2/10 [00:13<00:54,  6.78s/it]

Epoch 2/10 - loss: 0.1306936824926087 - val_auc: 0.8918814912429671 - val_rmse: 0.3689809424518295


Training epoch:  30%|███       | 3/10 [00:20<00:47,  6.73s/it]

...

Epoch 8/10 - loss: 0.11024483061861408 - val_auc: 0.9053429440798433 - val_rmse: 0.34327707410154545


Training epoch:  90%|█████████ | 9/10 [00:59<00:06,  6.44s/it]

Epoch 9/10 - loss: 0.10925354608658962 - val_auc: 0.9057030147563483 - val_rmse: 0.3420430198203784


Training epoch: 100%|██████████| 10/10 [01:05<00:00,  6.59s/it]

Epoch 10/10 - loss: 0.10845647188537122 - val_auc: 0.9059576675350323 - val_rmse: 0.3410580442946167
sparsity:  0.001778654950320264


In [21]:
pd.DataFrame(X_train)[0].value_counts().mean()

12.611678648001009

In [22]:
rating_sampled = rating.sample(10_000, random_state=42)

# Preprocess the data
merged = pd.merge(anime, rating_sampled, on="anime_id", suffixes=["_anime", "_user"])
merged = merged.dropna()
merged = merged[
    ["anime_id", "name", "genre", "type", "episodes", "rating_user", "user_id"]
]

# Create features from user and anime
user_enc = LabelEncoder()
anime_enc = LabelEncoder()
merged["user"] = user_enc.fit_transform(merged["user_id"].values)
merged["anime"] = anime_enc.fit_transform(merged["anime_id"].values)

# Transform ratings to binary
merged["rating"] = [1 if x >= 6 else 0 for x in merged["rating_user"]]

# Prepare the data
X = merged[["user", "anime"]].values
y = merged["rating"].values


# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the FM model
fm = FM(n_factors=20, learning_rate=0.01, n_epochs=30)
fm.fit(X_train, y_train, X_val, y_val)


n_users = int(np.max(X_train[:, 0]) + 1)
n_items = int(np.max(X_train[:, 1]) + 1)

print("sparsity: ", rating_sampled.shape[0] / (n_users * n_items))

Training epoch:   7%|▋         | 2/30 [00:00<00:01, 14.85it/s]

Epoch 1/30 - loss: 0.17896276075115986 - val_auc: 0.5213486013205321 - val_rmse: 0.44371553866886226
Epoch 2/30 - loss: 0.1704888519134453 - val_auc: 0.5425217696405218 - val_rmse: 0.44240248015869144
Epoch 3/30 - loss: 0.16306499966540552 - val_auc: 0.5580287710120889 - val_rmse: 0.44137525441410524


Training epoch:  20%|██        | 6/30 [00:00<00:01, 12.37it/s]

Epoch 4/30 - loss: 0.1564585873202387 - val_auc: 0.56969028101177 - val_rmse: 0.4405369609240656
Epoch 5/30 - loss: 0.15051408598645713 - val_auc: 0.5784032407259736 - val_rmse: 0.43983279201065684
Epoch 6/30 - loss: 0.14512006384487272 - val_auc: 0.5852011100124398 - val_rmse: 0.43922869154933847


Training epoch:  27%|██▋       | 8/30 [00:00<00:01, 13.27it/s]

...

Epoch 22/30 - loss: 0.09630662228950564 - val_auc: 0.6170163631144142 - val_rmse: 0.43469830940681337
Epoch 23/30 - loss: 0.09451674837199214 - val_auc: 0.6178201652259896 - val_rmse: 0.4345524331531713
Epoch 24/30 - loss: 0.09281104047052745 - val_auc: 0.6183509297949028 - val_rmse: 0.4344143961973805
Epoch 25/30 - loss: 0.09118376510515437 - val_auc: 0.6189110395202705 - val_rmse: 0.43428360875853633


Training epoch:  93%|█████████▎| 28/30 [00:02<00:00, 14.77it/s]

Epoch 26/30 - loss: 0.08962971782813926 - val_auc: 0.6195119772894007 - val_rmse: 0.4341595458617981
Epoch 27/30 - loss: 0.08814416156209195 - val_auc: 0.6198628432904851 - val_rmse: 0.4340417379292712
Epoch 28/30 - loss: 0.08672277360525645 - val_auc: 0.620337469299225 - val_rmse: 0.4339297630169558


Training epoch: 100%|██████████| 30/30 [00:02<00:00, 13.54it/s]

Epoch 29/30 - loss: 0.0853615998777897 - val_auc: 0.6207113010749259 - val_rmse: 0.43382324036849007
Epoch 30/30 - loss: 0.08405701525175381 - val_auc: 0.6211246850180218 - val_rmse: 0.4337218250294395
sparsity:  0.00042599559750589803


In [23]:
pd.DataFrame(X_train)[0].value_counts().mean()

1.1484352569623888